In [42]:
import fitz

def read_pdf_file(file_path, start_page=1, end_page=100):
    pdf_document = fitz.open(file_path)
    total_pages = pdf_document.page_count
    pdf_text = ''

    for page_num in range(start_page - 1, end_page):
        pdf_page = pdf_document.load_page(page_num)
        pdf_text += pdf_page.get_text()

    return pdf_text

year = 2021

file_path = f'elkem_data/pdf/{year}.pdf'
start_page=6
end_page=9
pdf_data = read_pdf_file(file_path,
                         start_page=start_page,
                         end_page=end_page
                         )

with open(f'elkem_data/raw/{year}.txt', 'w') as outfile:
    outfile.write(pdf_data)
    
print(pdf_data)


 
Konsernets resultatregnskap
Beløp i: NOK
Note
2021
2020
RESULTATREGNSKAP
Inntekter
Revenue
33 083 000 000
24 025 000 000
Other operating income
586 000 000
631 000 000
Share of profit (loss) from equity accounted companies
48 000 000
35 000 000
Sum inntekter
33 717 000 000
24 691 000 000
Kostnader
Raw materials and energy for production
15 861 000 000
12 858 000 000
Employee benefit expenses
4 530 000 000
4 028 000 000
Amortisation and depreciation
1 816 000 000
1 710 000 000
Impairment losses
76 000 000
17 000 000
Other operating expenses
5 536 000 000
5 121 000 000
Other items
113 000 000
130 000 000
Sum kostnader
27 932 000 000
23 864 000 000
Driftsresultat
5 785 000 000
827 000 000
Finansinntekter og finanskostnader
Share of profit (loss) from equity accounted financial investments
37 000 000
-15 000 000
Finance income
40 000 000
32 000 000
Sum finansinntekter
77 000 000
17 000 000
Finance expenses
276 000 000
278 000 000
Foreign exchange gains (losses)
-241 000 000
-18 000 000
S

In [173]:
import re
import pandas as pd

path = 'elkem_data/raw/2021.txt'
year = f'{path.split("/")[-1].split(".")[0]}'

data = open('elkem_data/raw/2021.txt', 'r').read()


# Extract Income Statement data
income_statement = re.search(r'(?<=RESULTATREGNSKAP)([\s\S]*?)(?=rsregnskap regnskaps)', data)

if income_statement:
    income_statement = income_statement.group().strip()

# Extract Balance Sheet data
balance_sheet = re.search(r'(?<=BALANSE - EIENDELER)([\s\S]*?)(?=SUM EGENKAPITAL OG GJELD)', data)




if balance_sheet:
    balance_sheet = balance_sheet.group().strip()

# Helper function to format extracted data
# def format_data(extracted_data):

    
#     data = {}

#     lines = extracted_data.split('\n')
    
#     for line in lines:
        
#         numerical_data = re.findall(r'(-?\d{1,3}(?:\s\d{3})*\s\d{3})', line)
#         numerical_data = ' '.join(numerical_data).replace(' ', '')
#         numerical_data = int(numerical_data) if numerical_data else None
        
        
    
#         if numerical_data:
    
            
#             next_line = lines[lines.index(line) + 1]
            
#             if re.findall(r'(\d{1,3}(?:\s\d{3})*\s\d{3})', next_line):
                
#                 previous_line = lines[lines.index(line) - 1]
#                 # print(previous_line)
#                 # print(numerical_data)
                
#                 data[previous_line] = numerical_data
            
#     df = pd.DataFrame(list(data.items()), columns=['Category', 'Amount'])

#     return data

import re
import pandas as pd

def format_data(extracted_data):
    data = {}
    lines = extracted_data.split('\n')
    
    for i, line in enumerate(lines):
        numerical_data = re.findall(r'(-?\d{1,3}(?:\s\d{3})*\s\d{3})', line)
        numerical_data = ' '.join(numerical_data).replace(' ', '')
        numerical_data = int(numerical_data) if numerical_data else None
        
        if numerical_data:
            # Check if the previous line exists and does not contain a number
            if i > 0 and not re.search(r'\d', lines[i-1]):
                variable_name = lines[i-1]
            # Check if the next line exists and does not contain a number
            elif i < len(lines) - 1 and not re.search(r'\d', lines[i+1]):
                variable_name = lines[i+1]
            # else:
            #     variable_name = f'Unknown Variable {len(data) + 1}'
            
            data[variable_name] = numerical_data
            
    # split the dictionary into two dictionaries one fore income statement and one for balance sheet
    # split on the value årsresultat, which is the last value in the income statement
    
    split_key = 'Årsresultat'
    keys = list(data.keys())
    split_index = keys.index(split_key) + 1; 
    
    income_statement, balance_sheet = {k: data[k] for k in keys[:split_index]}, {k: data[k] for k in keys[split_index:]}

    
    df = pd.DataFrame(list(data.items()), columns=['Category', 'Amount'])
    return data



#data=format_data(income_statement)

data=format_data(data)


data



{'Revenue': 33083000000,
 'Other operating income': 586000000,
 'Share of profit (loss) from equity accounted companies': 48000000,
 'Sum inntekter': 33717000000,
 'Kostnader': 24691000000,
 'Raw materials and energy for production': 15861000000,
 'Employee benefit expenses': 4530000000,
 'Amortisation and depreciation': 1816000000,
 'Impairment losses': 76000000,
 'Other operating expenses': 5536000000,
 'Other items': 113000000,
 'Sum kostnader': 27932000000,
 'Driftsresultat': 5785000000,
 'Finansinntekter og finanskostnader': 827000000,
 'Share of profit (loss) from equity accounted financial investments': 37000000,
 'Finance income': 40000000,
 'Sum finansinntekter': 77000000,
 'Finance expenses': 276000000,
 'Foreign exchange gains (losses)': -241000000,
 'Sum finanskostnader': 35000000,
 'Netto finans': 42000000,
 'Ordinært resultat før skattekostnad': 5827000000,
 'Income tax (expense) benefit': 1163000000,
 'Ordinært resultat etter skattekostnad': 4664000000,
 'Årsresultat': 2